<a href="https://colab.research.google.com/github/foreverchnf/GLOUNNLP-Bert/blob/master/bert_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

安装cuda

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!sudo apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda

--2020-07-11 12:59:47--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2020-07-11 12:59:47 (4.60 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

--2020-07-11 12:59:53--  http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:80... connected.
HTTP request sent, awaiting response...

安装mxnet和gluonnlp

In [3]:
!pip install --upgrade mxnet-cu100==1.6.0 -f https://dist.mxnet.io/python
!pip install --upgrade gluonnlp

Looking in links: https://dist.mxnet.io/python
     |████████████████████████████████| 695.0MB 27kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 256kB 7.1MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470053 sha256=30f749416d18aba924c50f49219d1a6fb52addf8c8e97e47632aee9a8a533851
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp


In [4]:
import os
os.chdir("/content/drive/My Drive/sentence_embedding")
!ls

aclImdb			elmo_sentence_representation.ipynb
aclImdb_v1.tar.gz	index.rst
attention-nlp.png	labeledTrainData.tsv
bert			log_MRPC.txt
bert-embed.png		log_SST.txt
bert_imdb.ipynb		log_STS-B.txt
bert.ipynb		output_dir
bert.png		self_attentive_sentence_embedding.ipynb
bert-sentence-pair.png	testDataset.tsv
Bi-LSTM-Rep.png		Untitled
dev.tsv			Untitled0.ipynb


导入必要的库

In [5]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector
# this notebook assumes that all required scripts are already
# downloaded from the corresponding tutorial webpage on http://gluon-nlp.mxnet.io

from bert import data

nlp.utils.check_version('0.8.1')

设置环境

In [6]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

导入BERT模型

In [7]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                        dataset_name='book_corpus_wiki_en_uncased',
                        pretrained=True, ctx=ctx, use_pooler=True,
                        use_decoder=False, use_classifier=False)
print(bert_base)

Vocab file is not found. Downloading.
BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): DotProductSelfAttentionCell(
          (dropout_layer): Dropout(p = 0.1, axes=())
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
      )
      (1): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (at

将模型用于分类

In [8]:
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

导入数据集

In [9]:
tsv_file = io.open('labeledTrainData.tsv', encoding='utf-8')
for i in range(5):
    print(tsv_file.readline())


id	sentiment	review

"5814_8"	1	"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feat

数据预处理

In [10]:
# Skip the first line, which is the schema
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
sample_splitter = nlp.data.Splitter('\n')
# Fields to select from the file
field_indices = [2,1]
data_train_raw = nlp.data.TSVDataset(filename='labeledTrainData.tsv',
                  sample_splitter=sample_splitter,
                  field_separator=field_separator,
                  num_discard_samples=num_discard_samples,
                  field_indices=field_indices)
sample_id = 100
# Sentence
print(data_train_raw[sample_id][0])
# label
print(data_train_raw[sample_id][1])


"There is a uk edition to this show which is rather less extravagant than the US version. The person concerned will get a new kitchen or perhaps bedroom and bathroom and is wonderfully grateful for what they have got. The US version of this show is everything that reality TV shouldn't be. Instead of making a few improvements to a house which the occupants could not afford or do themselves the entire house gets rebuilt. I do not know if this show is trying to show what a lousy welfare system exists in the US or if you beg hard enough you will receive. The rather vulgar product placement that takes place, particularly by Sears, is also uncalled for. Rsther than turning one family in a deprived area into potential millionaires, it would be far better to help the community as a whole where instead of spending the hundreds of thousands of dollars on one home, build something for the whole community ..... perhaps a place where diy and power tools can be borrowed and returned along with build

In [11]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence
max_len = 128

# The labels for the two classes [(0 = not similar) or  (1 = similar)]
all_labels = ["0", "1"]

# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = False
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                        class_labels=all_labels,
                        has_label=True,
                        pad=True,
                        pair=pair)
data_train = data_train_raw.transform(transform)

print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('segment ids = \n%s'%data_train[sample_id][1])
print('valid length = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])

vocabulary used for tokenization = 
Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = 
[    2  1000  2045  2003  1037  2866  3179  2000  2023  2265  2029  2003
  2738  2625 27856  2084  1996  2149  2544  1012  1996  2711  4986  2097
  2131  1037  2047  3829  2030  3383  5010  1998  5723  1998  2003  6919
  2135  8794  2005  2054  2027  2031  2288  1012  1996  2149  2544  1997
  2023  2265  2003  2673  2008  4507  2694  5807  1005  1056  2022  1012
  2612  1997  2437  1037  2261  8377  2000  1037  2160  2029  1996 18837
  2071  2025  8984  2030  2079  3209  1996  2972  2160  4152  7183  1012
  1045  2079  2025  2113  2065  2023  2265  2003  2667  2000  2265  2054
  1037 10223  6508  7574  2291  6526  1999  1996  2149  2030  2065  2017
 11693  2524  2438  2017  2097  4374  1012  1996  2738 29364  4031 11073
  2008  3138  2173  1010  3391  2011 18493     3]
segment ids = 
[0 0 0 0 0 0 0 0 0

In [13]:
!nvidia-smi -pm 1

Enabled persistence mode for GPU 00000000:00:04.0.
All done.


In [15]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17773046905027788525, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4852259398666451099
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7184977172667612689
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9715849664
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4911344454789088947
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

微调

In [16]:
# The hyperparameters
batch_size = 32
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_train],
                      batch_size=batch_size,
                      shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 100
num_epochs = 3
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(bert_dataloader):
        #mx.attach_grad()
        with mx.autograd.record():

        # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

        # Forward computation
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss = step_loss + ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                .format(epoch_id, batch_id + 1, len(bert_dataloader),
                step_loss / log_interval,
                trainer.learning_rate, metric.get()[1]))
            step_loss = 0

MXNetError: ignored

In [ ]:
nlp.utils.save_parameters(bert_classifier,bert_classifier)

测试

In [ ]:
type(data_train_raw[sample_id][0])
tsv_try = io.open('try0.tsv', encoding='utf-8')
print(tsv_try.readline())
print(tsv_try.readline())
data_try_row = nlp.data.TSVDataset(filename='try0.tsv',
                sample_splitter=sample_splitter,
                field_separator=field_separator,
                )
data_try=data_try_row.transform(transform)
print('token ids = \n%s'%data_try[0][0])
print('segment ids = \n%s'%data_try[0][1])
print('valid length = \n%s'%data_try[0][2])

try_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_try],
                      batch_size=batch_size,
                      shuffle=False)
try_dataloader = mx.gluon.data.DataLoader(data_try, batch_sampler=try_sampler)



In [ ]:
ctx=mx.gpu(0)
for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(try_dataloader):

        # Load the data to the GPU
        token_ids = token_ids.as_in_context(ctx)
        valid_length = valid_length.as_in_context(ctx)
        segment_ids = segment_ids.as_in_context(ctx)

        # Forward computation
        out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
        
        print(out)


In [ ]:
#type(out)
#out.shape
#mm=mx.ndarray.max(out,2)
print("sentence","polarity")
output=out.argmax(axis=1)
for item in range(len(output)):
  if(mm[item]==1):
    print(" ",item,"     positive")
  else:
    print(" ",item,"     negative")

测试集

In [ ]:
tsv_file = io.open('testDataset.tsv', encoding='utf-8')
for i in range(5):
    print(tsv_file.readline())


In [ ]:
field_indices=[2,1]
data_test_raw = nlp.data.TSVDataset(filename='testDataset.tsv',
                  sample_splitter=sample_splitter,
                  field_separator=field_separator,
                  num_discard_samples=num_discard_samples,
                  field_indices=field_indices)
sample_id = 100


In [ ]:
# Sentence
print(data_test_raw[sample_id])

In [ ]:
data_test = data_test_raw.transform(transform)

print('token ids = \n%s'%data_test[sample_id][0])
print('segment ids = \n%s'%data_test[sample_id][1])
print('valid length = \n%s'%data_test[sample_id][2])

In [ ]:
test_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_test],
                      batch_size=batch_size,
                      shuffle=True)
test_dataloader = mx.gluon.data.DataLoader(data_test, batch_sampler=test_sampler)

i = 0
err = 0
total = 0
for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(test_dataloader):
        i += 1

        # Load the data to the GPU
        token_ids = token_ids.as_in_context(ctx)
        valid_length = valid_length.as_in_context(ctx)
        segment_ids = segment_ids.as_in_context(ctx)
        label = label.as_in_context(ctx)

        # Forward computation
        out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
        output=out.argmax(axis=1)
        j = 0
        for lb in label:
            if lb == output[j].astype('int32'):
                err += 1
            j += 1
            total += 1
        # if i < 2:
        #     print(label)
        #     print(output)


In [ ]:
print('accuracy:',err/total)